In [32]:
import urllib.request
import xml.etree.ElementTree as ET
import datetime
import time
import json
import redis

In [33]:
INFLUXDB_HOST="10.10.0.214:8086"
#INFLUXDB_HOST="10.10.0.208:8086"
INFLUXDB_DATABASE="cityzone"
ACCESS_TOKEN="4f55964d-d361-44fe-94d8-2f2eaedce16e"
DATE_TIME=int(time.time()) * 1000000000
TIMEOUT=10
postUrl = "http://{}/write?db={}".format(INFLUXDB_HOST, INFLUXDB_DATABASE)

In [34]:
pool = redis.ConnectionPool(host='testgate.zan-qian.com', port=6379, password="640f9791f9d73033UG", decode_responses=True)
r = redis.Redis(connection_pool=pool)
info = r.info()

{'slave_expires_tracked_keys': 0, 'arch_bits': 64, 'used_memory_peak': 1823128, 'loading': 0, 'rdb_last_cow_size': 0, 'pubsub_channels': 0, 'aof_enabled': 0, 'used_memory_lua_human': '269.00K', 'atomicvar_api': 'atomic-builtin', 'pubsub_patterns': 0, 'aof_rewrite_scheduled': 0, 'active_defrag_hits': 0, 'used_memory_startup': 786576, 'blocked_clients': 0, 'db0': {'expires': 58, 'keys': 143, 'avg_ttl': 251830368}, 'client_longest_output_list': 0, 'uptime_in_seconds': 9339661, 'instantaneous_input_kbps': 0.0, 'lazyfree_pending_objects': 0, 'role': 'master', 'redis_git_dirty': 0, 'sync_partial_err': 0, 'active_defrag_key_misses': 0, 'repl_backlog_size': 1048576, 'maxmemory_policy': 'allkeys-lru', 'run_id': 'eb900751d90d26305952fd648139b7e5f0446095', 'os': 'Linux 4.4.0-117-generic x86_64', 'multiplexing_api': 'epoll', 'used_memory_rss_human': '5.39M', 'maxmemory': 2147483648, 'total_system_memory_human': '7.80G', 'rdb_last_save_time': 1530868106, 'sync_partial_ok': 0, 'master_replid2': 0, '

In [35]:
def sendMetric(info, name):
    value = info.get(name)
    data = "redisInfo,name={} value={} {}".format(name, value, DATE_TIME)
    urllib.request.urlopen(url = postUrl, data = data.encode('utf-8'))

In [38]:
def sendDbMetric(info):
    dbName = "db0"
    value = info.get(dbName)
    for key in value:
        data = "redisInfo,name={} value={} {}".format(dbName + "." + key, value[key], DATE_TIME)
        urllib.request.urlopen(url = postUrl, data = data.encode('utf-8'))


In [39]:
names = ["connected_clients","blocked_clients","used_memory","used_memory_rss","used_memory_peak","used_memory_lua","total_connections_received","total_commands_processed","instantaneous_ops_per_sec","total_net_input_bytes","total_net_output_bytes","keyspace_hits","keyspace_misses","used_cpu_sys","used_cpu_user"]
for name in names:
    sendMetric(info, name)

sendDbMetric(info)